In [126]:
from pulp import *
import pandas as  pd
import numpy as np

In [127]:
snp = pd.read_csv('snp_beta.csv')

snp.tail()

,0,%abs_return,beta
499,XYL,186.985627,1.20
500,YUM,1157.342959,0.50
501,ZBH,329.580033,1.14
502,ZION,18.336509,1.56
503,ZTS,126.296885,0.87


In [128]:
print(len(snp))

snp.replace('-', np.nan, inplace=True)

snp.dropna(inplace=True)

print(len(snp))

504
476


In [129]:
prob = LpProblem('Captial Allocater', LpMaximize)

In [130]:
decision_variables = []
for i in snp['0']:
    variables = pulp.LpVariable(str(i),lowBound = 0)
    decision_variables.append(variables)
print(decision_variables)

[MMM, ABT, ABBV, ABMD, ACN, ATVI, ADBE, AMD, AAP, AES, AMG, AFL, A, APD, AKAM, ALK, ALB, ARE, ALXN, ALGN, ALLE, AGN, ADS, LNT, ALL, GOOGL, MO, AMZN, AAL, AEP, AXP, AIG, AMT, AWK, AMP, ABC, AME, AMGN, APH, ADI, ANSS, ANTM, AON, AOS, APA, AIV, AAPL, AMAT, APTV, ADM, ARNC, AJG, AIZ, ATO, T, ADSK, ADP, AZO, AVB, AVY, BLL, BAC, BK, BAX, BBT, BDX, BBY, BIIB, BLK, HRB, BA, BWA, BXP, BSX, BMY, AVGO, BR, COG, CDNS, CPB, COF, CAH, KMX, CCL, CAT, CBS, CE, CELG, CNC, CNP, CTL, CERN, CF, SCHW, CHTR, CVX, CMG, CB, CHD, CI, XEC, CINF, CTAS, CSCO, C, CFG, CTXS, CLX, CME, CMS, KO, CTSH, CL, CMCSA, CMA, CAG, CXO, COP, ED, STZ, COO, CPRT, GLW, COST, COTY, CCI, CSX, CMI, CVS, DHI, DHR, DRI, DVA, DE, DAL, XRAY, DVN, FANG, DLR, DFS, DISCA, DISH, DG, DLTR, D, DOV, DTE, DUK, DRE, DXC, ETFC, EMN, ETN, EBAY, ECL, EIX, EW, EA, EMR, ETR, EOG, EFX, EQIX, EQR, ESS, EL, ES, RE, EXC, EXPE, EXPD, EXR, XOM, FFIV, FB, FAST, FRT, FDX, FIS, FITB, FE, FRC, FISV, FLT, FLIR, FLS, FMC, F, FTNT, FTV, FBHS, BEN, FCX, GPS, GRMN,

In [131]:
# decision_variables = []
# snp_list = snp['0'].to_list()
# variables = pulp.LpVariable.dicts('vars', snp_list, 0)
# print(variables)

In [132]:
#objective function
total_cost = ""
for variable, rtrn in zip(decision_variables, snp['%abs_return']):
    formula = variable*float(rtrn)/100
    total_cost+= formula 
# print(str(total_cost))
prob += total_cost

In [133]:
#constrain 1
total_cap = 100000
sum_capital = ""
for variable in decision_variables:
    formula  = variable
    sum_capital += formula

prob += (sum_capital <= total_cap)

In [136]:
#constrain 2
total_cost = ""
portfolio_Beta = 0.1
for variable, beta in zip(decision_variables, snp['beta']):
    mul = abs(float(beta)) - portfolio_Beta
    formula = variable*float(mul)
    total_cost+= formula

prob += (total_cost <= 0)

In [137]:
print(prob)

Captial Allocater:
MAXIMIZE
2.42027311613664*A + -0.46672950741972324*AAL + 5.650399019541018*AAP + 115.99885850507967*AAPL + 1.0989948145820314*ABBV + 5.863969791790055*ABC + 10.188656716417908*ABMD + 2.4047601595481027*ABT + 6.309043594218458*ACN + 9.916569067675077*ADBE + 1.7662319499675083*ADI + 2.857581372721412*ADM + 2.5101057557516318*ADP + 12.479358758277169*ADS + 10.46853695872464*ADSK + 2.492207066975313*AEP + -0.14585309937309224*AES + 3.9821506762885726*AFL + 4.152032025691723*AGN + -0.9492884898193701*AIG + 1.9403653853853315*AIV + 3.157059003768729*AIZ + 2.288701294932868*AJG + 9.84*AKAM + 12.686530247631113*ALB + 46.937432578209275*ALGN + 9.686276668062453*ALK + 3.5996946417957143*ALL + 0.8459170497895065*ALLE + 18.685596707818927*ALXN + 2.091862282854207*AMAT + -0.37659187386294724*AMD + 16.000477778744642*AME + 3.4056305809250627*AMG + 2.39180159052684*AMGN + 4.324462878114628*AMP + 15.911560847485255*AMT + 104.64317976513101*AMZN + 20.970971343505767*ANSS + 9.37656072

In [138]:

optimization_result = prob.solve()
name = []
capital = []
assert optimization_result == pulp.LpStatusOptimal
print("Status:", LpStatus[prob.status])
print("Optimal Solution to the problem: ", value(prob.objective))
print ("Individual decision_variables: ")
for v in prob.variables():
# 	print(v.name, "=", v.varValue)
    name.append(v.name)
    capital.append(v.varValue)
sol = pd.DataFrame({'name':name, 'capital':capital})

print(sol[sol['capital'] != 0])

Status: Optimal
Optimal Solution to the problem:  7026774.084174188
Individual decision_variables: 
     name     capital
294  MNST   8547.0085
454   WEC  91452.9910
